In [1]:
from api.services.similarity_recommender_service import SimilarityRecommenderService
from api.services.Recomender import recomend_me
service = SimilarityRecommenderService()
service.init()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gillesdeknache/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gillesdeknache/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
query = "Leonardo"

m1 = service.recommend(query)
m2 = recomend_me(query, top_k= 3)

In [3]:
m1

[('The Revenant  (2015)', 4),
 ('The Wolf of Wall Street  (2013)', 3),
 ('Inception (2010)', 3)]

In [16]:
L = []
for m in m1:
    a = m[0].rsplit(' ',1)[0].strip()
    L.append((a,m[1]))


L

[('The Revenant', 4), ('The Wolf of Wall Street', 3), ('Inception', 3)]

In [17]:
m2

(['Shutter Island', 'Titanic', 'The Departed'],
 [0.22160209763092573, 0.2113956480183037, 0.208104277409475])

In [26]:
from collections import defaultdict

score = defaultdict(lambda:0)

In [27]:
total = 0 
for movie in L:
    total += movie[1]
total

10

In [28]:
for movie in L:
    score[movie[0]] = movie[1]/total
score

defaultdict(<function __main__.<lambda>()>,
            {'The Revenant': 0.4,
             'The Wolf of Wall Street': 0.3,
             'Inception': 0.3})

In [30]:
for movie,score2 in zip(m2[0],m2[1]):
    score[movie] += score2 * 1.3

score

defaultdict(<function __main__.<lambda>()>,
            {'The Revenant': 0.4,
             'The Wolf of Wall Street': 0.3,
             'Inception': 0.3,
             'Shutter Island': 0.5540052440773143,
             'Titanic': 0.5284891200457593,
             'The Departed': 0.5202606935236875})

In [35]:
sorted_score = sorted(score.items(), key=lambda x:x[1], reverse=True)
sorted_score

[('Shutter Island', 0.5540052440773143),
 ('Titanic', 0.5284891200457593),
 ('The Departed', 0.5202606935236875),
 ('The Revenant', 0.4),
 ('The Wolf of Wall Street', 0.3),
 ('Inception', 0.3)]

In [38]:
k = 3 
recommended_movies = []
for movie in sorted_score[:k]:
    recommended_movies.append(movie[0])

recommended_movies

['Shutter Island', 'Titanic', 'The Departed']

In [53]:
def mixture_recommend(query):
    m1 = service.recommend(query,k=3)
    m2 = recomend_me(query, top_k= 3)
    L = []
    for m in m1:
        a = m[0].rsplit(' ',1)[0].strip()
        L.append((a,m[1]))
    score = defaultdict(lambda:0)
    total = 0 
    for movie in L:
        total += movie[1]
    for movie in L:
        score[movie[0]] = movie[1]/total
    for movie,score2 in zip(m2[0],m2[1]):
        score[movie] += score2 * 1.3

    sorted_score = sorted(score.items(), key=lambda x:x[1], reverse=True)
    k = 3 
    recommended_movies = []
    for movie in sorted_score[:k]:
        recommended_movies.append(movie[0])
    return recommended_movies

In [55]:
mixture_recommend("Natalie Portman drama")

['Black Swan', 'V for Vendetta', 'American Made']